In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
from utils import gauss_interp
from get_data import get_memristor_data

In [ ]:
path = 'Data/Partial_Reset_PCM.pkl'
n_mem = 1
vs_data, mus_data, sigs_data = get_memristor_data(path,n_mem)

n_samp, n_m = vs_data.shape

In [ ]:
# vs_data = np.arange(0, 10, 1)[:, np.newaxis]

In [ ]:
# mus_data = np.zeros_like(vs_data)
# mus_data[4:6, :] = 1

In [ ]:
interp_width = np.array(vs_data[1, 0] - vs_data[0, 0]).astype('float32')

In [ ]:
d_in = 784 
n_layers = 3 
f = lambda x: x  # Nonlinearity
f = tf.nn.tanh

interp_width = np.array(vs_data[1, 0] - vs_data[0, 0]).astype('float32')
GAMMA = 1.0  # slope of the out of bounds cost

in_shape = (None, d_in)
noise_shape = (None, n_m)

n_h_e = 200 
n_h_d = 200 

learning_rate = 0.001

In [ ]:
VMIN, VMAX = vs_data.min(), vs_data.max()


In [ ]:
# Input
x = tf.placeholder(tf.float32, shape=in_shape)

# Noise
eps = tf.placeholder(tf.float32, shape=noise_shape)

# Values for memristors
vs = tf.Variable(initial_value=vs_data.astype('float32'), trainable=False, name='vs')
mus = tf.Variable(initial_value=mus_data.astype('float32'), trainable=False, name='mus')
sigs = tf.Variable(initial_value=sigs_data.astype('float32'), trainable=False, name='sigs')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
v = tf.placeholder(tf.float32, (None, n_m))

In [ ]:
t_mu = gauss_interp(v, vs, mus, interp_width=interp_width, ratio=1.)

In [ ]:
batch_size = 5000

In [ ]:
vs_data.shape

In [ ]:
vs_data[:, 0]

In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
config = tf.ConfigProto(gpu_options=gpu_options)

with tf.Session(config=config) as sess:
    sess.run(tf.initialize_all_variables())
    eps_values = np.random.randn(batch_size, n_m) 
    v_values = np.random.rand(batch_size, n_m) * (VMAX - VMIN) + VMIN
    mu_values = sess.run(t_mu, feed_dict={v: v_values,
                              eps: eps_values})

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(v_values, mu_values)
plt.scatter(vs_data, mus_data, c='g')